<a href="https://colab.research.google.com/github/guilhermelaviola/IntegratingPracticesInDataScienceForBusiness/blob/main/Class16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd

# Fictitious supplier data:
data = {
    'Supplier': ['Supplier A', 'Supplier B', 'Supplier C'],
    'Cost ($)': [12000, 13500, 12500],  # Lower is better
    'Quality (1-10)': [8.5, 7.0, 9.0],   # Higher is better
    'Delivery Time (days)': [15, 20, 12], # Lower is better
    'Support (1-10)': [7, 9, 6]           # Higher is better
}

df = pd.DataFrame(data)

# Displaying the DataFrame:
df.head()

,Supplier,Cost ($),Quality (1-10),Delivery Time (days),Support (1-10)
0,Supplier A,12000,8.5,15,7
1,Supplier B,13500,7.0,20,9
2,Supplier C,12500,9.0,12,6


In [3]:
# Creating a normalized DataFrame:
normalized_df = df.copy()

# Min-max normalization:
def normalize(series, minimize=False):
    if minimize:
        return (series.max() - series) / (series.max() - series.min())
    else:
        return (series - series.min()) / (series.max() - series.min())

normalized_df['Cost (norm)'] = normalize(df['Cost ($)'], minimize=True)
normalized_df['Quality (norm)'] = normalize(df['Quality (1-10)'])
normalized_df['Delivery (norm)'] = normalize(df['Delivery Time (days)'], minimize=True)
normalized_df['Support (norm)'] = normalize(df['Support (1-10)'])

# Displaying the normalized DataFrame:
normalized_df.head()

,Supplier,Cost ($),Quality (1-10),Delivery Time (days),Support (1-10),Cost (norm),Quality (norm),Delivery (norm),Support (norm)
0,Supplier A,12000,8.5,15,7,1.000000,0.75,0.625,0.333333
1,Supplier B,13500,7.0,20,9,0.000000,0.00,0.000,1.000000
2,Supplier C,12500,9.0,12,6,0.666667,1.00,1.000,0.000000


## **Quantitative techniques**

In [4]:
from numpy import linalg as LA
import numpy as np

# Pairwise comparison matrix (Cost, Quality, Delivery, Support):
ahp_matrix = np.array([
    [1,   1/3, 3,   1],
    [3,   1,   5,   3],
    [1/3, 1/5, 1,   1/2],
    [1,   1/3, 2,   1]
])

# Normalizing and calculating priority vector (weights):
col_sums = ahp_matrix.sum(axis=0)
normalized = ahp_matrix / col_sums
weights = normalized.mean(axis=1)

criteria = ['Cost', 'Quality', 'Delivery', 'Support']
ahp_weights = dict(zip(criteria, weights))

In [5]:
# Applying weights:
normalized_df['AHP Score'] = (
    normalized_df['Cost (norm)'] * ahp_weights['Cost'] +
    normalized_df['Quality (norm)'] * ahp_weights['Quality'] +
    normalized_df['Delivery (norm)'] * ahp_weights['Delivery'] +
    normalized_df['Support (norm)'] * ahp_weights['Support']
)

ranked_df = normalized_df.sort_values('AHP Score', ascending=False)

In [6]:
# PROMETHEE - Simple Preference Function Example by subtracting average dominated score from dominating score:
from sklearn.preprocessing import MinMaxScaler

# Simple weighted PROMETHEE net flow example:
weighted_matrix = normalized_df[['Cost (norm)', 'Quality (norm)', 'Delivery (norm)', 'Support (norm)']].copy()

for i, crit in enumerate(criteria):
    weighted_matrix[crit] *= weights[i]

normalized_df['PROMETHEE Score'] = weighted_matrix.sum(axis=1)

KeyError: 'Cost'

## **Qualitative example**